In [26]:
import dash
from dash import Dash,html,dcc,Input,Output,callback
import dash_bootstrap_components as dbc
from dash import dash_table as dt
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go

In [10]:
#intputs
PNi = dbc.InputGroup(
    [dbc.InputGroupText("Project Name"), dbc.Input(id='PN',placeholder="Project Name")],
    className="mb-3")
TRi = dbc.InputGroup(
    [dbc.InputGroupText("Top Reservoir"), dbc.Input(id='TR',value="2100",type="number"),dbc.InputGroupText("TVDMSL")],
    className="mb-3")
BRi = dbc.InputGroup(
    [dbc.InputGroupText("Base Reservoir"), dbc.Input(id='BR',value="2300",type="number"),dbc.InputGroupText("TVDMSL")],
    className="mb-3")
NTGi = dbc.InputGroup(
    [dbc.InputGroupText("NTG"), dbc.Input(id='NTG',value="0.88",type="number"),dbc.InputGroupText("Fraction")],
    className="mb-3")
WDi = dbc.InputGroup(
    [dbc.InputGroupText("Water depth"), dbc.Input(id='WD',value="150",type="number"),dbc.InputGroupText("m")],
    className="mb-3")
RRi = dbc.InputGroup(
    [dbc.InputGroupText("Reservoir Radius"), dbc.Input(id='RR',value="3500",type="number"),dbc.InputGroupText("m")],
    className="mb-3")
Di = dbc.InputGroup(
    [dbc.InputGroupText("Depletion"), dbc.Input(id='D',value="13",type="number"),dbc.InputGroupText("bar")],
    className="mb-3")
Ei = dbc.InputGroup(
    [dbc.InputGroupText("Youngs Modulus, E"), dbc.Input(id='E',value="11",type="number"),dbc.InputGroupText("GPa")],
    className="mb-3")
nui = dbc.InputGroup(
    [dbc.InputGroupText("Poissons Ratio, nu"), dbc.Input(id='nu',value="0.12",type="number")],
    className="mb-3")
ei = dbc.InputGroup(
    [dbc.InputGroupText("End of data range"), dbc.Input(id='e',value="8000",type="number"),dbc.InputGroupText("m")],
    className="mb-3")
intvi = dbc.InputGroup(
    [dbc.InputGroupText("Data interval"), dbc.Input(id='intv',value="100",type="number"),dbc.InputGroupText("m")],
    className="mb-3")

In [11]:
def calcvariables(TR,BR,NTG,WD,Ew,nuw,RR):
    TR = float(TR)
    BR =float(BR)
    NTG =float(NTG)
    WD =float(WD)
    Ew =float(Ew)
    nuw =float(nuw)
    RR =float(RR)

    RT = (float(BR)-float(TR))*float(NTG)
    OB = float(TR)-float(WD)
    K = float(Ew)/(3*(1-2*float(nuw)))
    C = OB/float(RR)
    M = (float(Ew)*(1-float(nuw)))/((1-(2*float(nuw)))*(1+float(nuw)))
    B = 1-(K/37)
    Cm = 1/M/1000*B
    data = {
        'Calculated variable':  ['Reservoir thickness','Overburden thickness','Bulk Modulus, K','C (depth/radius)','Uniaxial Compaction Modulus, M','Biot','Cm'],
        'Value': [RT,OB,K,C,M,B,Cm],
        'Units': ['m','m','GPa','fraction','GPa','','MPa^*1'],
    }
    df = pd.DataFrame(data)
    df['Value'] = pd.to_numeric(df['Value'], errors='coerce') # formatting the value column as numbers
    df.loc[df.index[:-1], 'Value'] = df['Value'][:-1].apply(lambda x: '{:.2f}'.format(x)) # formatting value column to 2 sig figs apart from the last value
    last_row_index = len(df) - 1
    last_cell_value = df.at[last_row_index, 'Value'] 
    df.at[last_row_index, 'Value'] = '{:.2e}'.format(float(last_cell_value)) # formating last value to be a scientific type number with 2 sig figs
    columns = [{'name': col, 'id': col} for col in df.columns] # creating column variable to output into a datatable in the dash app
    data = df.to_dict('records')

    return TR,BR,NTG,WD,Ew,nuw,RR,RT,OB,K,C,M,B,Cm,data,columns

In [12]:
def geertsma(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D):
    TR, BR, NTG, WD, Ew, nuw,RR, RT,OB,K,C,M,B,Cm,data,columns = calcvariables(TR,BR,NTG,WD,Ew,nuw,RR)
    e = float(e)
    intv=float(intv)
    D=float(D)
    # create dataframe for a range of data points defined by the water depth (first depth) and depth of interest (number of data points controlled by input intv)
    df_results = pd.DataFrame((np.arange(WD,e,intv)), columns=['Depth'])   
    df_results['Z (m)'] = ((df_results['Depth']) - WD)/TR   # Creating normalised depth
    
    #creating conditions to asign data to OB, UB or reservoir
    conditions = [
        (df_results['Depth'] < TR),
        (df_results['Depth'] >= TR) & (df_results['Depth'] <= BR),
        (df_results['Depth'] > BR)
        ]
    df_results['definition'] = np.select(conditions, ['Overburden','Reservoir','Underburden'])   # defining data points as either overburden, reservoir and underburden
    
    # Calculation of Uz (broken into smaller componants)
    Z = df_results['Z (m)']
    a = (Cm/2)*RT*D*B
    b = (C*(Z-1))/pow(1+C*C*(Z-1)*(Z-1),(1/2))
    c = ((3-(4*nuw))*C*(Z+1))/pow(1+C*C*(Z+1)*(Z+1),(1/2))
    d = (2*C*Z)/pow((1+C*C*(Z+1)*(Z+1)),(3/2))
    e_OB = 3-(4*nuw)+1   # +1 used if overburden
    e_UB = 3-(4*nuw)-1   # -1 used if underburden
    conditions = [
        (df_results['definition'] == 'Overburden'),
        (df_results['definition'] == 'Reservoir'),
        (df_results['definition'] == 'Underburden')
        ]
    Uz_OB = a*(b-c+d+e_OB)
    Uz_UB = a*(b-c+d+e_UB)
    df_results['Uz (m)'] = np.select(conditions, [Uz_OB,Uz_OB,Uz_UB])   # calculation of Uz
    df_results['Uz (cm)'] = df_results['Uz (m)'] * 100
    
    Uz_end = df_results['Uz (m)'].iloc[-1]    # Extracting the end Uz value
    df_results['Vertical displacement (m)'] = df_results['Uz (m)'] - Uz_end    # Calculation of vertical displacement
    df_results['Vertical displacement (cm)'] = df_results['Vertical displacement (m)']*100

    df_maxD = df_results['Vertical displacement (cm)'].idxmax()     #finding index of row with max displacement - corresponds to top reservoir
    df_next = df_maxD + 1                                               #creating the index of next row after max displacement row
    df_results_compaction = df_results.loc[df_maxD:df_next]         # new dataframe with data at the top and base reservoir

    Vd_OB = df_results_compaction['Vertical displacement (cm)'].iloc[0]     # extracting the vertical displacement just above res
    Vd_UB = df_results_compaction['Vertical displacement (cm)'].iloc[-1]    # extracting the vertical displacement just beow the res
    Compaction = Vd_OB - Vd_UB   #calculating reservoir compaction
    Compaction_est = RT*D*Cm*100 
    sb_sub = df_results.iloc[0,6]    # sea bed subsidence is the first displacement value calculated for seabed depth

    Res_strain = ((Compaction/10) / RT)*100
    OB_strain = ((sb_sub/10)/ OB)*100

    Compaction = "{:.2f}".format(Compaction)
    sb_sub ="{:.2f}".format(sb_sub)
    Res_strain = "{:.2f}".format(Res_strain)
    OB_strain = "{:.2f}".format(OB_strain)
    
    sub = f'Seabed subsidence: {sb_sub} cm'
    compaction = f'Reservoir compaction: {Compaction} cm'
    strain_res = f'Reservoir strain: {Res_strain} %'
    strain_ob = f'Overburden strain: {OB_strain} %'
    
    return df_results, sub,compaction, Compaction, sb_sub, strain_res,strain_ob

In [13]:
def graph(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D):
    df_results,sub,compaction, Compaction, sb_sub,strain_res,strain_OB = geertsma(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D)
    
    maxD = df_results['Vertical displacement (cm)'].max()
    minD = df_results['Vertical displacement (cm)'].min()
        
    x_range = [(minD-2),(maxD+2)]
    Seabed_y = (WD,WD)
    Res_top_y = (TR,TR)
    Res_base_y = (BR, BR)

    depthdata = {
        'xrange':[(minD-2),(maxD+2)],
        'Seabed_y': [WD,WD],
        'Res_top_y': [TR,TR],
        'Res_base_y': [BR, BR]
    }

    depth_df = pd.DataFrame(depthdata)

    profile = go.Scatter(
        x= df_results['Vertical displacement (cm)'],
        y= df_results['Depth'],
        mode='lines',
        name='Displacement profile',
        line=dict(color='black')
    )

    sb = go.Scatter(
        x= depth_df['xrange'],
        y= depth_df['Seabed_y'],
        mode='lines',
        name='Seabed',
        line=dict(color='steelblue')
    )

    rest = go.Scatter(
        x= depth_df['xrange'],
        y= depth_df['Res_top_y'],
        mode='lines',
        name='Reservoir Top',
        line=dict(color='gold')
    )

    resb = go.Scatter(
        x= depth_df['xrange'],
        y= depth_df['Res_base_y'],
        mode='lines',
        name='Reservoir Base',
        line=dict(color='darkorange')
    )

    layout = go.Layout(
        xaxis=dict(title='Vertical displacement (cm)',side='top', mirror=True, title_standoff=5,title_font=dict(size=12, family='arial'),zeroline=True, zerolinewidth=1, zerolinecolor='grey'),
        yaxis=dict(title='Depth (mTVDMSL)',autorange='reversed',title_font=dict(size=12, family='arial'), title_standoff=5,zeroline=True, zerolinewidth=1, zerolinecolor='grey',mirror=True),
        legend=dict(orientation='h', x=0.1, y=0.16,bordercolor='black',borderwidth=1),
        height=800,
        plot_bgcolor='white',
        template='simple_white' 
        
    )

    fig = go.Figure(data=[profile,sb,rest,resb], layout=layout)

    return fig

In [14]:
def format_numeric(val):
    if isinstance(val, (int, float)):
        return round(val, 2)
    return val

In [15]:
# building app
app = dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])

app.layout = html.Div(
    [dbc.Row([
        dbc.Col([
            dbc.Row( html.H5('User inputs',style={'font-family':'arial'})),
            dbc.Row(PNi),
            dbc.Row(TRi),
            dbc.Row(BRi),
            dbc.Row(NTGi),
            dbc.Row(WDi),
            dbc.Row(RRi),
            dbc.Row(Di),
            dbc.Row(Ei),
            dbc.Row(nui),
            dbc.Row(html.H6('Define data limits',style={'font-family':'arial'})),
            dbc.Row(ei),
            dbc.Row(intvi, style={'margin-bottom': '10px'}),
            dbc.Row(html.H5("Calculated variables:",style={'font-family':'arial','marginTop': '10px'})),
            dbc.Row(dt.DataTable(id='dt1', columns=[],style_as_list_view=True, style_cell={'textAlign': 'left','font-family': 'Arial','padding': '8px'},style_header={ 'display': 'none' ,'marginTop': '0px'},style_table={'marginTop': '0px'},
                     style_cell_conditional=[{'if': {'column_id': 'Calculated variable'},'textAlign': 'right'}]))
        ],width=3),        
        dbc.Col([
            dbc.Row(html.H5('Calculated subsidence/compaction',style={'font-family':'arial'} )),
            dbc.Row(html.Div(id='sub',style={'fontWeight': 'bold','font-family':'arial'})),
            dbc.Row(html.Div(id='compaction', style={'fontWeight': 'bold','font-family':'arial','marginBottom': '15px'})),
            dbc.Row(html.Div(id='strain_OB',style={'fontWeight': 'bold','font-family':'arial'})),
            dbc.Row(html.Div(id='strain_res',style={'fontWeight': 'bold','font-family':'arial','marginBottom': '15px'})),
            
            dbc.Row(html.H5('Raw data',style={'font-family':'arial','margintop': '10'})),
            dbc.Row(dbc.Checklist(id='toggle-button',inline=True,options=[{'label': 'Toggle to show raw data table', "value": 1}], value=[],switch=True)),

            dbc.Row(html.H5('Sensitivity table',style={'font-family':'arial','margintop': '10','marginTop': '15px'})),
            dbc.Row(html.P('Populate a sensitivity table to compare results for different inputs',style={'font-family':'arial','margintop': '0','marginTop': '0px'})),
            dbc.Row(dbc.InputGroup([dbc.Input(id='SensL',placeholder="Add label to sensitivity data e.g. Base Case")],className="mb-3")),
            dbc.Row([
                html.Div([dbc.Button("Add current data to sensitivity table", id='add-row', style={'font-family':'arial','marginTop':'5px','marginRight':'10px'}, className="d-grid gap-2 col-8", size="sm")])
            ]),
            dbc.Row(dcc.Graph(id='graph')),
        ],width=4),
        dbc.Col([
            dbc.Row(html.H5('Sensitivity table',style={'font-family':'arial'} )),
            dbc.Row([
                html.Div([dbc.Button("Save sensitivity data", id='save-sens', style={'font-family':'arial','marginTop':'5px','marginRight':'10px','marginBottom': '15px'}, className="d-grid gap-2 col-8", size="sm")])
            ]),
            dbc.Row([
                dt.DataTable(id='sensitivity',columns=[{'name': col, 'id': col} for col in ['Label','Reservoir thickness, m', 'OB thickness, m','Youngs Modulus, GPa','Poissons Ratio','Seabed subsidence, cm','Res compaction, cm']], data=[],
                             style_cell={'textAlign': 'center', 'padding': '5px', 'fontFamily': 'Arial, sans-serif'},
                             style_header={'whiteSpace': 'normal', 'height': 'auto','fontFamily': 'Arial, sans-serif'}
                            ),
                dcc.Store(id='button-clicks', data=0)
            ])
        ],width=5),
    ]),
     dbc.Col([
         html.Div(id='output-table-container',style={'font-family':'arial','margin-bottom': '0','margin-top': '10'}, children=[
             html.Hr(style={'borderWidth': "0.5vh"}),
             html.H5("Calculated subsidence data", style={'marginTop': '10px', 'marginBottom': '5px'}),
             dt.DataTable(id='output-table', columns=[],style_table={'width': '400px', 'marginTop': '0px', 'marginBottom': '0px'},style_cell={'textAlign': 'center', 'padding': '5px', 'fontFamily': 'Arial, sans-serif'},merge_duplicate_headers=True),
         ]),
     ],width=12),
])

@callback(
    Output('dt1', 'columns'),
    Output('dt1', 'data'),
    Output('sub', 'children'),
    Output('compaction', 'children'),
    Output('strain_res', 'children'),
    Output('strain_OB', 'children'),
    Output('graph', 'figure'),
    Input(component_id='TR', component_property='value'),
    Input(component_id='BR', component_property='value'),
    Input(component_id='NTG', component_property='value'),
    Input(component_id='WD', component_property='value'),
    Input(component_id='E', component_property='value'),
    Input(component_id='nu', component_property='value'),
    Input(component_id='RR', component_property='value'),
    Input(component_id='e', component_property='value'),
    Input(component_id='intv', component_property='value'),
    Input(component_id='D', component_property='value'),
)

def appcalc(TR,BR,NTG,WD,Ew,nuw,RR,e,intv,D):
    TR, BR, NTG, WD, Ew, nuw,RR, RT,OB,K,C,M,B,Cm,data,columns = calcvariables(TR,BR,NTG,WD,Ew,nuw,RR)
    df_results,sub,compaction,Compaction, sb_sub,strain_res,strain_OB = geertsma(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D)
    fig = graph(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D)

    return columns,data,sub,compaction,strain_res,strain_OB, fig, #df_results

@app.callback(
    Output('output-table-container', 'style'),
    Output('output-table', 'columns'),
    Output('output-table', 'data'),
    #Output('dataframe-store', 'data'),
    Input('toggle-button', 'value'),
    Input(component_id='TR', component_property='value'),
    Input(component_id='BR', component_property='value'),
    Input(component_id='NTG', component_property='value'),
    Input(component_id='WD', component_property='value'),
    Input(component_id='E', component_property='value'),
    Input(component_id='nu', component_property='value'),
    Input(component_id='RR', component_property='value'),
    Input(component_id='e', component_property='value'),
    Input(component_id='intv', component_property='value'),
    Input(component_id='D', component_property='value'),
)

def toggle_output_table(toggle_value, TR,BR,NTG,WD,Ew,nuw,RR,e,intv,D):
    # Determine visibility of the output table based on the toggle button's value
    table_visibility = {'display': 'block' if 1 in toggle_value else 'none'}
    
    df_results,sub,compaction,Compaction, sb_sub,strain_res,strain_OB = geertsma(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D)
    numeric_columns = df_results.columns.difference(['definition'])
    df_results[numeric_columns] = df_results[numeric_columns].applymap(lambda x: '{:.2f}'.format(x) if isinstance(x, (int, float)) else x)
    
    # Create columns and data for the output table
    columns = [{"name": col, "id": col} for col in df_results.columns]
    Gdata = df_results.to_dict('records')
    
    return table_visibility, columns, Gdata, 

@app.callback(
    Output("button-clicks", "data"), 
    Input("add-row", "n_clicks")
)

def update_button_clicks(n_clicks):
    if n_clicks is None:
        return 0
    return n_clicks

@app.callback(
    Output('sensitivity','data'),
    Input('button-clicks', 'data'),
    State('sensitivity','data'),
    State(component_id='TR', component_property='value'),
    State(component_id='BR', component_property='value'),
    State(component_id='NTG', component_property='value'),
    State(component_id='WD', component_property='value'),
    State(component_id='E', component_property='value'),
    State(component_id='nu', component_property='value'),
    State(component_id='RR', component_property='value'),
    State(component_id='e', component_property='value'),
    State(component_id='intv', component_property='value'),
    State(component_id='D', component_property='value'),
    State(component_id='SensL', component_property='value')
)

def sensitivityTable(n_clicks,current_data,TR,BR,NTG,WD,Ew,nuw,RR,e,intv,D,SensL):
    if n_clicks == 0:
        return dash.no_update
        
    TR, BR, NTG, WD, Ew, nuw,RR, RT,OB,K,C,M,B,Cm,data,columns = calcvariables(TR,BR,NTG,WD,Ew,nuw,RR)
    df_results,sub,compaction,Compaction, sb_sub,strain_res,strain_OB = geertsma(TR, BR, NTG, WD, Ew, nuw,RR, e,intv,D)

    new_row_data = {
        'Label':SensL,
        'Reservoir thickness, m':RT, 
        'OB thickness, m':OB,
        'Youngs Modulus, GPa':Ew,
        'Poissons Ratio':nuw,
        'Seabed subsidence, cm':sb_sub,
        'Res compaction, cm':Compaction
    }        
    
    sensitivity_updated_data = current_data + [new_row_data] 
    
    return sensitivity_updated_data
'''
@app.callback(
    Output('sensitivity', 'data'),
    Input('save-sens', 'n_clicks'),
    State('sensitivity', 'data'),
    prevent_initial_call=True  # Prevent the initial call from firing
)
def save_table_as_excel(n_clicks, sens_data):
    if not n_clicks:
        raise dash.exceptions.PreventUpdate  # Prevent the callback from running
    
    if not sens_data:
        raise dash.exceptions.PreventUpdate 
    
    df = pd.DataFrame(sens_data)
    df.to_excel('table_data.xlsx', index=False)  # Save the data as an Excel file
    
    return dash.no_update
'''

if __name__ == '__main__':
    app.run_server(port=8057,debug=True) 
    